In [10]:
import torch
import os
import shutil
import pandas as pd
import numpy as np

# --- 제공해주신 코드의 클래스들을 임포트합니다 ---

# 모델 관련
from models.model import (GATree, GATreePop, FEATURE_NUM, FEATURE_PAIR, ALL_FEATURES,
                   COL_NODE_TYPE, NODE_TYPE_UNUSED)

# 변이 연산자 관련
from evolution.Mutation.base import BaseMutation
from evolution.Mutation.chain import ChainMutation
from evolution.Mutation.node_param import NodeParamMutation
from evolution.Mutation.reinitialize_node import ReinitializeNodeMutation
from evolution.Mutation.add_node import AddNodeMutation
from evolution.Mutation.delete_node import DeleteNodeMutation
from evolution.Mutation.add_subtree import AddSubtreeMutation
from evolution.Mutation.delete_subtree import DeleteSubtreeMutation

import data.data_download as data_dd
import data.feature_generator as data_fe

In [41]:
# df = data_dd.fetch_historical_data('BTCUSDT', '1m', 14)

df = pd.read_csv('dataset/btc_1m_t.csv')
df['Close time'] = pd.to_datetime(df['Close time'])
df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Close time']]

In [42]:
# 1. 추세 (Trend) 지표
df, price_col = data_fe.calculate_price_change_features(df)
df, ma_col = data_fe.calculate_ma(df, window=14)
df, ema_col = data_fe.calculate_ema(df, window=24)
df, vwma_col = data_fe.calculate_vwma(df, window=5)
df, macd_cols = data_fe.calculate_macd(df, short_window=12, long_window=26, signal_window=9)
df, adx_cols = data_fe.calculate_adx(df, window=14)
# Ichimoku는 기본값(9, 26, 52)을 사용합니다.
df, ichimoku_cols = data_fe.calculate_ichimoku(df) 

# 2. 모멘텀 (Momentum) 지표
df, rsi_col = data_fe.calculate_rsi(df, window=14)
df, stoch_cols = data_fe.calculate_stochastic_oscillator(df, k_window=14, d_window=3)
df, williams_r_col = data_fe.calculate_williams_r(df, window=14)
# CCI는 기본 상수값(0.015)을 사용합니다.
df, cci_col = data_fe.calculate_cci(df, window=20)

# 3. 변동성 (Volatility) 지표
# Bollinger Bands는 기본 표준편차(2)를 사용합니다.
df, bb_cols = data_fe.calculate_bollinger_bands(df, window=20, num_std=2)
df, atr_col = data_fe.calculate_atr(df, window=14)

# 4. 거래량 (Volume) 지표
df, obv_col = data_fe.calculate_obv(df)
df, cmf_col = data_fe.calculate_cmf(df, window=20)

# 5. 시간 기반 피처
# 이 함수를 사용하려면 'Timestamp' 컬럼이 datetime 형식이어야 합니다.
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df, time_day_cols = data_fe.cyclic_encode_fn(df, timestamp_col='Close time', cycle='minute_of_day')
df, time_week_cols = data_fe.cyclic_encode_fn(df, timestamp_col='Close time', cycle='day_of_week')

# 7. 지지 (Support) 및 저항 (Resistance) 지표
df, sr_cols = data_fe.calculate_support_resistance(df, window=14)

In [52]:
cci_col

['CCI_20_0.015']